# 1Password Security Auditor Expert - Exemplo de Uso

Este notebook demonstra como usar a ferramenta de auditoria de segurança do 1Password.

## 1. Setup

Instalar dependências necessárias:

In [ ]:
!pip install ollama huggingface_hub

## 2. Autenticação 1Password

Verificar se 1Password CLI está autenticado:

In [ ]:
import subprocess
import json

# Verificar autenticação
result = subprocess.run(
    ["op", "whoami", "--format", "json"],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    user = json.loads(result.stdout)
    print(f"✅ Autenticado como: {user.get('email')}")
else:
    print("❌ Não autenticado. Execute: eval $(op signin)")

## 3. Listar Cofres Disponíveis

In [ ]:
# Listar cofres
result = subprocess.run(
    ["op", "vault", "list", "--format", "json"],
    capture_output=True,
    text=True
)

vaults = json.loads(result.stdout)
print("📦 Cofres disponíveis:")
for vault in vaults:
    print(f"  - {vault['name']} ({vault['id']})")

## 4. Exportar Items de um Cofre

In [ ]:
# Selecionar cofre
VAULT_NAME = "1p_vps"  # Altere conforme necessário

# Exportar items
result = subprocess.run(
    ["op", "item", "list", "--vault", VAULT_NAME, "--format", "json"],
    capture_output=True,
    text=True
)

items = json.loads(result.stdout)
print(f"📊 Total de items em {VAULT_NAME}: {len(items)}")

# Exibir primeiros 5 items
for item in items[:5]:
    print(f"  - {item['title']} ({item['category']})")

## 5. Executar Auditoria com CLI

In [ ]:
import sys
sys.path.append("../cli")

# Importar módulo de auditoria
from audit_1password_expert import export_vault, build_audit_prompt

# Exportar cofre
vault_data = export_vault(VAULT_NAME, include_secrets=False)

print(f"✅ Exportado: {vault_data['total_items']} items")

## 6. Analisar Nomenclatura

In [ ]:
import re

# Padrão SSOT v2.1
SSOT_PATTERN = r"^(PROD|DEV|SHARED|MACOS|VPS|AZURE)_[A-Z0-9_]+$"

violations = []
compliant = []

for item in vault_data["items"]:
    title = item["title"]
    if re.match(SSOT_PATTERN, title):
        compliant.append(title)
    else:
        violations.append(title)

print(f"✅ Conformes: {len(compliant)} ({len(compliant)/len(vault_data['items'])*100:.1f}%)")
print(f"❌ Violações: {len(violations)}")

if violations:
    print("\nExemplos de violações:")
    for title in violations[:5]:
        print(f"  - {title}")

## 7. Analisar Tags

In [ ]:
# Tags obrigatórias
REQUIRED_SCOPE_TAGS = ["production", "development", "staging", "global"]
REQUIRED_TYPE_TAGS = ["database", "api_key", "service_account", "ssh_key", "oauth", "certificate"]

missing_tags = []

for item in vault_data["items"]:
    tags = [t.lower() for t in item.get("tags", [])]
    
    has_scope = any(t in tags for t in REQUIRED_SCOPE_TAGS)
    has_type = any(t in tags for t in REQUIRED_TYPE_TAGS)
    
    if not has_scope or not has_type:
        missing_tags.append({
            "title": item["title"],
            "tags": tags,
            "missing_scope": not has_scope,
            "missing_type": not has_type
        })

print(f"❌ Items sem tags obrigatórias: {len(missing_tags)}")

if missing_tags:
    print("\nExemplos:")
    for item in missing_tags[:3]:
        print(f"  - {item['title']}")
        print(f"    Tags atuais: {item['tags']}")
        if item['missing_scope']:
            print(f"    Falta: tag de escopo")
        if item['missing_type']:
            print(f"    Falta: tag de tipo")

## 8. Gerar Comandos de Correção

In [ ]:
print("📝 Comandos de correção:\n")

# Correção de nomenclatura
for title in violations[:3]:
    suggested = f"PROD_{title.upper().replace(' ', '_').replace('-', '_')}"
    item_id = next((i["id"] for i in vault_data["items"] if i["title"] == title), "ID")
    print(f"# {title}")
    print(f"op item edit {item_id} 'title={suggested}' --vault {VAULT_NAME}")
    print()

# Correção de tags
for item in missing_tags[:3]:
    item_id = next((i["id"] for i in vault_data["items"] if i["title"] == item["title"]), "ID")
    print(f"# {item['title']}")
    print(f"op item edit {item_id} --tags 'production,database' --vault {VAULT_NAME}")
    print()

## 9. Próximos Passos

Para uma auditoria completa com LLM:

```bash
# Via CLI (Ollama local)
python ../cli/audit_1password_expert.py --vaults 1p_vps

# Via HuggingFace API
python ../cli/audit_1password_expert.py --vaults 1p_vps --hf

# Via Interface Web
cd ../gradio
python app.py
# Acesse: http://localhost:7860
```